# Equality Constrained Convex Quadratic Program Solvers

## Imports

In [29]:
#%conda install -c conda-forge cvxopt
#%conda install -c conda-forge casadi
#%conda install -c conda-forge matplotlib



In [30]:
import numpy as np
import scipy as sp
from scipy.sparse.linalg import splu, spsolve
from scipy.sparse import csc_matrix, csr_matrix, dia_matrix
import casadi as ca
from casadi import *
import matplotlib.pyplot as plt
import cvxopt as cvx
from cvxopt import cholmod


# Problem 1.3

The Goal here is to find x and y which solve the KKT matrix algebra problem:


$ \begin{bmatrix} H & -A \\ -A^T & 0 \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix} = \begin{bmatrix} -g \\ -b \end{bmatrix} $

This formulation is slightly different from what can be found on page 451 of the texbook. There are the following differences:    
* We define $A$ as the transpose of the textbook's definition: $A_{report} = A^T_{texbook}$
* We have some different variable names: $H_{report} = G_{textbook}$, and $g_{report} = c_{textbook}$
* We have a negative sign in from of the bottom left $A$ matrix, and in front of $b$.
    
    - This is permissible since $-Ax = -b$ implies $Ax = b$ and vice versa (that is to say, we are solving for the same $x$ and $y$).

    - This is required since we need the KKT matrix to be symmetric for factorization.                   

## Dense LU Factorization

In [31]:
def EqualityQPSolverLUdense(H,g,A,b):

    _, Am = A.shape #note that An = Hn = Hm, so we use Am for the dimensions of the 0 matrix
    KKT = np.block([[H, -A],[-A.T,np.zeros([Am,Am])]])
    L, U = sp.linalg.lu(KKT, permute_l = True)
    vec = -np.concatenate([g, b], axis=0) 
    midd = np.linalg.solve(L, vec)
    out_vec = np.linalg.solve(U, midd)

    xout = out_vec[:len(g)]
    yout = out_vec[len(g):]
    
    return xout, yout

## Sparse LU Factorization

In [32]:
def EqualityQPSolverLUsparse(H,g,A,b):

    _, Am = A.shape #note that An = Hn = Hm, so we use Am for the dimensions of the 0 matrix
    KKT = np.block([[H, -A],[-A.T,np.zeros([Am,Am])]])
    L, U = sp.linalg.lu(KKT, permute_l = True)
    L = csc_matrix(L) 
    U = csc_matrix(U)
    vec = csc_matrix(-np.concatenate([g, b], axis=0) )
    midd = sp.sparse.linalg.spsolve(L, vec)
    out_vec = np.array(sp.sparse.linalg.spsolve(U, midd))

    xout = out_vec[:len(g)]
    yout = out_vec[len(g):]
    
    return xout, yout

In [33]:
def EqualityQPSolverLUsparse_smart(H,g,A,b):

    _, Am = A.shape #note that An = Hn = Hm, so we use Am for the dimensions of the 0 matrix
    KKT = sp.sparse.csc_matrix(np.block([[H, -A],[-A.T,np.zeros([Am,Am])]])) #define the KKT matrix as a csc matrix
    LUsparse= splu(KKT) # the sparse lu function from scipy uses csc matrices. This produces an object with a .solve function
    vec = -np.concatenate([g, b], axis=0)
    out_vec = LUsparse.solve(vec) #using the .solve functionality to get the solution

    xout = out_vec[:len(g)]
    yout = out_vec[len(g):]
    
    return xout, yout

## Dense LDL.T Factorization

In [35]:
def EqualityQPSolverLDLdense(H,g,A,b):

    _, Am = A.shape 
    KKT = np.block([[H, -A],[-A.T,np.zeros([Am,Am])]]) #note that An = Hn = Hm, so we use Am for the dimensions of the 0 matrix
    L, D, perm = sp.linalg.ldl(KKT) #scipy's linalg.ldl performs a dense LDL.T decomposition
    vec = -np.concatenate([g, b], axis=0) 
    midd1 = np.linalg.solve(L, vec) #here we solve the system using the LDL.T factorization
    midd2 = np.linalg.solve(D, midd1)
    out_vec = np.linalg.solve(L.T, midd2) 
    xout = out_vec[:len(g)]
    yout = out_vec[len(g):]
    
    return xout, yout

## Sparse LDL.T Factorization

In [36]:
def EqualityQPSolverLDLsparse(H,g,A,b):

    _, Am = A.shape 
    KKT = np.block([[H, -A],[-A.T,np.zeros([Am,Am])]]) #note that An = Hn = Hm, so we use Am for the dimensions of the 0 matrix
    L, D, perm = sp.linalg.ldl(KKT) #scipy's linalg.ldl performs a dense LDL.T decomposition
    L = csc_matrix(L)
    D = csc_matrix(D)

    vec = csc_matrix(-np.concatenate([g, b], axis=0)) 
    midd1 = sp.sparse.linalg.spsolve(L, vec) #here we solve the system using the LDL.T factorization
    midd2 = sp.sparse.linalg.spsolve(D, midd1)
    out_vec = sp.sparse.linalg.spsolve(L.T, midd2) 
    xout = out_vec[:len(g)]
    yout = out_vec[len(g):]
    
    return xout, yout

In [ ]:
def EqualityQPSolveLDLsparse2(H,g,A,b): #borken
    An, Am = A.shape #note that An = Hn = Hm, so we use Am for the dimensions of the 0 matrix
    KKT_np = np.block([[H, -A],[-A.T,np.zeros([Am,Am])]])
    print(KKT_np)
    print(sp.linalg.det(KKT_np))
    KKT = sp.sparse.csc_matrix(KKT_np) #we make the KKT sparse in preparation for converting to cvxopt
    # For the sparse LDL solver we utilize cvxopt. However, first we must convert our scipy sparse KKT matrix to a cvxopt sparse matrix
    # since cvxopt uses Fortran (column) indexing, this requires some maneuvering
    data = KKT.data
    indices = KKT.indices 
    indptr = KKT.indptr
    I = []
    J = []
    V = []
    for col in range(KKT.shape[1]):
        for idx in range(indptr[col], indptr[col+1]):
            I.append(int(indices[idx]))
            J.append(int(col))
            V.append(float(data[idx]))
    KKT_cvx = cvx.spmatrix(V, I, J, KKT.shape)
    print(KKT_cvx)
    vec = -np.concatenate([g, b], axis=0)
    print(vec)
    vec_cvx = cvx.matrix(vec)
    print(vec_cvx)
    # Symbolic analysis
    F = cholmod.symbolic(KKT_cvx)

    # Numeric factorization
    cholmod.numeric(KKT_cvx, F)

    # Solve A x = b
    out_vec = np.array(cholmod.solve(F, vec_cvx,sys=1))

    xout = out_vec[:len(g)]
    yout = out_vec[len(g):]
    
    return xout, yout

## Range Space Method
More info in lecture 5 QuadraticOptimization page 20/23.
Useful when H is easy to invert or known explicitly, and when there are few equality constraints.

In [40]:
def EqualityQPSolverRange(H,g,A,b):
    Hinv = sp.linalg.inv(H)
    A_T = A.T
    v = np.linalg.solve(H,g)
    y = np.linalg.solve((A_T@Hinv@A),b+(A_T@v))
    x = np.linalg.solve(H,(A@y-g))
    return x, y

## Null Space Method
Read more in lecture 5 QuadraticOptiization page 22/23. Useful when an-am is small.

In [140]:
def EqualityQPSolverNull(H,g,A,b):
    #This one is built from: Lecture 5 QuadraticOptimization page 22/23: https://learn.inside.dtu.dk/d2l/le/lessons/242131/topics/969401
    _, Am = A.shape 
    A_T = A.T
    Y,R = sp.linalg.qr(A,mode='economic') #This works but must be justified: goal: A^T@Y is nonsingular.
    Z = sp.linalg.null_space(A_T) #Note that we want A^T@Z=0, so Z is the null space of A^T
    Y_T = Y.T
    Z_T = Z.T
    x_Y = np.linalg.solve((A_T@Y),b)
    x_Z = np.linalg.solve((Z_T@H@Z),-Z_T@((H@Y@x_Y)+g))
    x = (Y@x_Y)+(Z@x_Z)
    y = np.linalg.solve((A_T@Y).T,Y_T@((H@x)+g))
    return x, y

# Problem 1.6
In this section we solve a provided test question using one of our QP solvers, and compare the solution which Casadi finds.

## Defining the Test Problem

In [119]:
#Defining the test problem for 1.6
H = np.array([
    [6.0000, 1.8600, 1.2400, 1.4800, -0.4600],
    [1.8600, 4.0000, 0.4400, 1.1200, 0.5200],
    [1.2400, 0.4400, 3.8000, 1.5600, -0.5400],
    [1.4800, 1.1200, 1.5600, 7.2000, -1.1200],
    [-0.4600, 0.5200, -0.5400, -1.1200, 7.8000]
])


g = np.array([
    [-16.1000],
    [-8.5000],
    [-15.7000],
    [-10.0200],
    [-18.6800]
])

A = np.array([
    [16.1000, 1.0000],
    [8.5000, 1.0000],
    [15.7000, 1.0000],
    [10.0200, 1.0000],
    [18.6800, 1.0000]
])


b = np.array([
    [15],
    [1]
])

_, Am = A.shape #note that An = Hn = Hm, so we use Am for the dimensions of the 0 matrix
KKT_np = np.block([[H, -A],[-A.T,np.zeros([Am,Am])]])


<h3>Casadi<h3>

In [42]:
#We need a converter to get to Casadi formatting
def QP_Convert_To_Casadi(Hin, gin, Ain, bin): #This function accepts H, g, A, and b, and outputs Casadi's inputs

    # Convert all arrays to CASADI types first
    Hout = ca.SX(Hin)
    gout = ca.SX(gin)

    # Build constraint matrix
    Aout = ca.SX(Ain.T)


    # Build bounds
    lbg = ca.DM(bin)
    ubg = ca.DM(bin)
    
    Hn, _ = Hin.shape
    lbx = ca.DM(np.full((1, Hn), -np.inf))
    ubx = ca.DM(np.full((1, Hn),  np.inf))
        
    return Hout, gout, Aout, lbx, ubx, lbg, ubg

In [43]:
#This function accepts H, g, A, and b, and returns the Casadi solution
def Casadi_EC_QP(Hin,gin,Ain,bin): #This function accepts the same initial data that our solver accepts, and then returns the Casadi solution.
    nh, _ = Hin.shape
        
    H, g, A, lbx_, ubx_, lbg_, ubg_ = QP_Convert_To_Casadi(Hin,gin,Ain,bin)

    #print("H shape:", H.shape)
    #print("g shape:",g.shape)
    #print("A shape:", A.shape)
    #print("lbg shape:", lbg_.shape)
    #print("ubg shape:", ubg_.shape)

    # Define symbolic decision variable
    x = ca.SX.sym('x', nh) 

    # Objective: minimize 0.5 * x^T H x + g^T x
    objective = 0.5 * ca.dot(x, ca.mtimes(H, x)) + ca.dot(g, x)

    # Constraint: 
    constraint = ca.mtimes(A, x)

    # Define the QP in symbolic form
    qp = {
        'x': x,       # decision variables
        'f': objective,
        'g': constraint
    }

    # Create solver
    solver = ca.qpsol('S', 'qpoases', qp)

    # Call solver with bounds (symbolic part already defined)
    sol = solver(
        lbx=lbx_, 
        ubx=ubx_,  
        lbg=lbg_,   # 
        ubg=ubg_  #   
    )

    return sol['x']

## Solving the Test Problem

In [129]:
#First we solve the test problem with one of our solvers:
x, y = EqualityQPSolverNull2(H,g,A,b)
print("x:\n",x.T,"\ny:\n",y.T)

#Next we do so with Casadi

Casadi_EC_QP(H,g,A,b)

Z_T: (0, 2)
H: (5, 5)
Y: (5, 2)


UnboundLocalError: cannot access local variable 'x_Y' where it is not associated with a value

## Comparison

Our dense LDL solver finds:

x = [0.15020779, 0.1400325, 0.35434944, 0.09364569, 0.26176457]

Casadi finds:

x = [0.150208, 0.140033, 0.354349, 0.0936457, 0.261765]

These solutions are satisfactorily similar.

In [ ]:
H = np.array([
    [6.0000, 1.8600, 1.2400, 1.4800, -0.4600],
    [1.8600, 4.0000, 0.4400, 1.1200, 0.5200],
    [1.2400, 0.4400, 3.8000, 1.5600, -0.5400],
    [1.4800, 1.1200, 1.5600, 7.2000, -1.1200],
    [-0.4600, 0.5200, -0.5400, -1.1200, 7.8000]
])


g = np.array([
    [-16.1000],
    [-8.5000],
    [-15.7000],
    [-10.0200],
    [-18.6800]
])

A = np.array([
    [16.1000, 1.0000],
    [8.5000, 1.0000],
    [15.7000, 1.0000],
    [10.0200, 1.0000],
    [18.6800, 1.0000]
])


b = np.array([
    [15],
    [1]
])

bn = 1000
b1_space = np.linspace(8.50,18.68,bn)
xsol_matrix = np.zeros((bn,5))

for b_id in range(len(b1_space)):
    b[0,0]=b1_space[b_id]
    xsol, _ = EqualityQPSolverLUdense(H,g,A,b)
    xsol_matrix[b_id,:] = xsol.flatten()



plt.figure(figsize=(10, 6))
for i in range(xsol_matrix.shape[1]):
    plt.plot(b1_space, xsol_matrix[:, i], label=f'x_{i + 1}')

plt.xlabel('b_1 value')
plt.ylabel('x values')
plt.title('x Solution Values as a Function of b_1')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'EqualityQPSolveLUdense' is not defined